In [1]:
from bs4 import BeautifulSoup

In [2]:
import pandas as pd

In [3]:
from urllib.request import urlopen
from tqdm.notebook import tqdm

# 날짜 지정

In [4]:
start = '2021-05-20'
end = '2021-05-21'

In [16]:
url_pt1 = 'https://www.hankookilbo.com/Search?Page='
url_pt2 = '&tab=NEWS&sort=recent&searchText=&searchTypeSet=TITLE,CONTENTS&realPaperYN=false&startDate='
url_pt3 = '%2000:00:00&endDate='
url_pt4 = '%2000:00:00&selectedPeriod=manual'

url = url_pt1 + '1' + url_pt2 + str(start) + url_pt3 + str(end) + url_pt4

In [17]:
# soup 지정
url_page = urlopen(url)
soup = BeautifulSoup(url_page,'html.parser')

# 페이지 수
pages = soup.find_all('span',class_='only-number')[0].string
pages = int(pages.replace(',',''))
pages = round(pages / 10 + 1)
pages

35

# 날짜, 제목, 링크 추출

In [25]:
# 날짜
soup.find_all('p','date')[5]

<p class="date">2021.05.20 22:02 최은서 기자 silver@hankookilbo.com</p>

In [26]:
# 제목 및 링크
soup.find_all(attrs={'target':'_self'})[5]

<a href="/News/Read/A2021052023080000356" target="_self">LG전에 좋은 기억을 가진 김영규(21ㆍNC)가 이번에도 팀의 ‘LG전 7연패’를 끊어내는데 앞장 섰다. NC는 20일 서울 잠실구장에서 열린 2021 KBO리그 LG와 경기에서 </a>

In [6]:
a_list = []
for p in tqdm(range(1,pages)):
    # 링크, soup
    url = url_pt1 + str(p) + url_pt2 + str(start) + url_pt3 + str(end) + url_pt4
    url_page = urlopen(url)
    soup = BeautifulSoup(url_page,'html.parser')
    
    # 제목, 링크
    title = soup.find_all(attrs={'target':'_self'})[::3]
    
    for i in range(len(title)):
        # 날짜
        tmp_d = soup.find_all('p','date')[i].string.split()[0]
        # 제목
        tmp_t = title[i].string
        tmp_l = title[i].attrs['href']
        # 링크
        tmp_l = 'https://www.hankookilbo.com/' + tmp_l
        a_list.append([tmp_d,tmp_t,tmp_l])

In [7]:
df = pd.DataFrame(a_list)
df.columns = ['date','title','link']   # 칼럼명 지정
df.head()

,date,title,link
0,2021.05.21,"아가야, 바다 건너 꼭 희망을 찾으렴",https://www.hankookilbo.com//News/Read/A202105...
1,2021.05.21,‘LG전 완봉의 추억’ 김영규 “오늘도 완투 욕심 있었지만…”,https://www.hankookilbo.com//News/Read/A202105...
2,2021.05.20,"김오수 청문회에 野 ""조국·서민 불러야""...與 ""불가""",https://www.hankookilbo.com//News/Read/A202105...
3,2021.05.20,"""중국 게 섯거라""… SK-포드 합작사 설립, 한미 배터리 동맹 가속화",https://www.hankookilbo.com//News/Read/A202105...
4,2021.05.20,"문 대통령, '강규형 전 KBS 이사 해임 부당' 판결에 불복해 상고",https://www.hankookilbo.com//News/Read/A202105...


# 본문 추출

In [28]:
url_page = urlopen(df.link[0])
soup = BeautifulSoup(url_page,'html.parser')    
soup.find_all('p','editor-p')

[<p class="editor-p">스페인령 세우타의 경비 조직 가르디아 시빌 대원이 18일 인근 해역에서 불법 이민자 부모로부터 분리된 갓난아기를 구조하고 있다. 최근 스페인 정부가 코로나19에 감염된 모로코 반군 지도자의 치료 목적 입국을 허가하자 이에 반발한 모로코 정부가 국경 경비를 느슨하게 하면서 아프리카를 떠나 유럽으로 향하는 대규모 불법 이민 행렬이 이어지고 있다. 가르디아 시빌 제공</p>]

In [9]:
df['article'] = ''
for i in tqdm(range(len(df))):
    url_page = urlopen(df.link[i])
    soup = BeautifulSoup(url_page,'html.parser')
    
    tmp = soup.find_all('p','editor-p')  # 본문
    text = ''
    for t in range(len(tmp)):
        text += tmp[t].get_text()
    df.article[i] = text

# 단어 추출

In [11]:
from konlpy.utils import pprint
from konlpy.tag import Komoran
import re
komoran = Komoran()
# 설치오류가 나는 경우, python버전과 맞는 JPype를 설치 후 진행한다(要 검색).

In [12]:
regex = '\d+월|^\d+$|\d+일|\d+년|^한국일보$'   # 특정 정규식 지정

df['words'] = ''
for i in tqdm(range(len(df))):
    # 명사 추출 후 set()으로 중복제거, sorted()로 오름차순 list화
    tmp = sorted(set(komoran.nouns(df.article[i])))
    
    # 지정한 특정 정규식과 일치하는 것을 삭제
    tmp = [re.sub(regex,'',s) for s in tmp]
    
    # 길이가 1이하인 list 요소 삭제
    tmp = [s for s in tmp if len(s) > 1]
    
    # '' 제거 후 df 행에 입력
    df.words[i] = ','.join(tmp)

In [14]:
df.to_csv('./data/한국일보(2.20~2.21).csv',encoding='utf-8-sig')

저장 후 해당 csv를 확인해보니, 지정된 날짜와 일치하지 않는 기사가 여러개 존재한다.
한국일보 사이트에서 날짜를 지정해보니 같은 오류가 발생해, 해당 사이트의 오류로 예상된다.